In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
from plug_mongo import get_entries
df = pd.DataFrame(get_entries()).drop(["_id", "timestamp"], axis=1).set_index("game_id")
df.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_8_w,player_8_l,player_8_co_w,player_8_co_l,player_8_ct_wr,player_9_w,player_9_l,player_9_co_w,player_9_co_l,player_9_ct_wr
game_id,,,,,,,,,,,,,,,,,,,,,
548809662,0,216,181,145,91,0.5354,174,147,8,3,...,297,284,1,0,0.5003,941,930,43,35,0.5093
4484824702,0,541,503,27,28,0.5161,160,111,0,0,...,269,202,9,3,0.4971,988,954,65,78,0.4948
548810655,1,155,146,20,6,0.4879,55,46,0,0,...,186,144,22,23,0.5088,55,45,0,1,0.5229
6140028214,1,686,645,43,41,0.5082,368,281,17,11,...,234,198,25,32,0.5197,2090,2035,390,340,0.5306
1351099000,1,31,14,23,8,0.5043,38,12,2,0,...,17,6,8,3,0.5148,125,103,0,0,0.4933


In [5]:
def add_one(column):
	return df[column].apply(lambda x: x+1)

def get_winrate(column):
	win = add_one(column+'_w')
	los = add_one(column+'_l')
	return (win / (win + los)).rename(column+"_wr").apply(lambda x: x - 0.5)

get_winrate('player_0').head()

game_id
548809662     0.043860
4484824702    0.018164
548810655     0.014851
6140028214    0.015379
1351099000    0.180851
Name: player_0_wr, dtype: float64

In [11]:
result = [df]
result.extend([get_winrate(f"player_{x}") for x in range(10)])
result.extend([get_winrate(f"player_{x}_co") for x in range(10)])
df_winrates = pd.concat(result, axis=1)
for x in range(10):
    df_winrates[f"player_{x}_ct_wr"] = df_winrates[f"player_{x}_ct_wr"].apply(lambda x: x - 0.5)
    df_winrates[f"player_{x}_w"] = np.log2(df_winrates[f"player_{x}_w"].apply(lambda x: x + 1))
    df_winrates[f"player_{x}_l"] = np.log2(df_winrates[f"player_{x}_l"].apply(lambda x: x + 1))
    df_winrates[f"player_{x}_co_w"] = np.log2(df_winrates[f"player_{x}_co_w"].apply(lambda x: x + 1))
    df_winrates[f"player_{x}_co_l"] = np.log2(df_winrates[f"player_{x}_co_l"].apply(lambda x: x + 1))
df_winrates.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_0_co_wr,player_1_co_wr,player_2_co_wr,player_3_co_wr,player_4_co_wr,player_5_co_wr,player_6_co_wr,player_7_co_wr,player_8_co_wr,player_9_co_wr
game_id,,,,,,,,,,,,,,,,,,,,,
548809662,0,7.761551,7.507795,7.189825,6.523562,0.0354,7.451211,7.209453,3.169925,2.000000,...,0.113445,0.192308,0.289474,-0.017241,0.200000,0.000000,0.084211,0.166667,0.166667,0.050000
4484824702,0,9.082149,8.977280,4.807355,4.857981,0.0161,7.330917,6.807355,0.000000,0.000000,...,-0.008772,0.000000,0.033537,0.227273,0.045763,-0.071429,0.166667,0.049020,0.214286,-0.044828
548810655,1,7.285402,7.199672,4.392317,2.807355,-0.0121,5.807355,5.554589,0.000000,0.000000,...,0.250000,0.000000,0.000000,-0.214286,-0.166667,0.000000,0.000000,0.000000,-0.010638,-0.166667
6140028214,1,9.424166,9.335390,5.459432,5.392317,0.0082,8.527477,8.139551,4.169925,3.584963,...,0.011628,0.100000,0.264706,0.125000,0.300000,0.142857,0.027538,0.106383,-0.059322,0.034153
1351099000,1,5.000000,3.906891,4.584963,3.169925,0.0043,5.285402,3.700440,1.584963,0.000000,...,0.227273,0.250000,-0.071429,0.000000,0.166667,0.031469,0.166667,0.000000,0.192308,0.000000


In [12]:
train_dataset = df_winrates.sample(frac=0.8, random_state=0)
test_dataset = df_winrates.drop(train_dataset.index)
train_dataset.head()

,label,player_0_w,player_0_l,player_0_co_w,player_0_co_l,player_0_ct_wr,player_1_w,player_1_l,player_1_co_w,player_1_co_l,...,player_0_co_wr,player_1_co_wr,player_2_co_wr,player_3_co_wr,player_4_co_wr,player_5_co_wr,player_6_co_wr,player_7_co_wr,player_8_co_wr,player_9_co_wr
game_id,,,,,,,,,,,,,,,,,,,,,
550397422,0,5.754888,5.807355,1.000000,0.000000,0.0017,5.832890,6.022368,1.000000,1.584963,...,0.166667,-0.100000,0.000000,0.166667,0.000000,0.055556,-0.250000,0.000000,0.083333,0.000000
1237347333,1,8.139551,8.027906,2.584963,3.169925,-0.0085,6.303781,6.442943,1.000000,2.321928,...,-0.100000,-0.214286,0.062500,0.100000,0.076000,0.100000,0.300000,0.154971,-0.055556,-0.166667
1323252065,1,7.888743,7.562242,4.906891,4.087463,0.0023,8.426265,8.262095,1.000000,0.000000,...,0.138298,0.166667,0.125000,-0.021739,0.078431,0.166667,0.000000,0.002000,-0.300000,0.121849
2626458901,0,5.321928,5.044394,5.169925,4.857981,0.0197,6.321928,6.266787,1.584963,2.000000,...,0.053846,-0.071429,0.071429,0.187500,-0.090909,0.071429,0.166667,-0.166667,-0.166667,0.100000
2631536191,1,6.700440,6.321928,5.459432,4.000000,0.0164,6.672425,6.820179,0.000000,0.000000,...,0.233333,0.000000,-0.023810,0.166667,0.100000,0.200000,0.250000,0.060000,0.000000,-0.071429


In [13]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('label')
test_labels = test_features.pop('label')
train_labels.head()

game_id
550397422     0
1237347333    1
1323252065    1
2626458901    0
2631536191    1
Name: label, dtype: int64

In [14]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(train_features)

In [15]:
def build_and_compile_model(norm, width, depth, activation='relu'):
    layer_l = [norm]
    layer_l.extend([layers.Dense(width, activation=activation) for _ in range(depth)])
    layer_l.append(layers.Dense(1))
    model = keras.Sequential(layer_l)
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [16]:
early_stop = keras.callbacks.EarlyStopping(patience=5, verbose=0, restore_best_weights=True)
activations = [None, 'relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh', 'selu', 'elu']

def test_model(width, depth, activation, verbose=1):
    with tf.device("GPU:0"):
        model = build_and_compile_model(normalizer, width, depth, activation)
        history_return = model.fit(
            train_features,
            train_labels,
            validation_split=0.2,
            verbose=verbose, epochs=100,
            callbacks = [early_stop]
        )
    return min(history_return.history['val_loss']), len(history_return.history['val_loss']), model

In [17]:
modes = []
for x in [35, 70, 140, 280]:
    for y in [2, 3, 4, 5, 6, 7, 8]:
        for z in ['softsign', 'sigmoid', 'softmax']:
            modes.append((x, y, z))
results = [(mode, test_model(mode[0], mode[1], mode[2], 0)) for mode in modes]

In [18]:
sorted_results = [(x[0], x[1][0], x[1][1]) for x in sorted(results, key=lambda x: x[1][0])]
sorted_results

[((35, 5, 'softsign'), 0.4512683153152466, 12),
 ((140, 5, 'softsign'), 0.45127061009407043, 15),
 ((70, 7, 'softsign'), 0.4517468214035034, 11),
 ((35, 6, 'softsign'), 0.45199790596961975, 12),
 ((35, 6, 'sigmoid'), 0.45230719447135925, 11),
 ((70, 6, 'sigmoid'), 0.45250430703163147, 16),
 ((140, 7, 'sigmoid'), 0.4529477655887604, 11),
 ((70, 6, 'softsign'), 0.4529983699321747, 29),
 ((140, 5, 'sigmoid'), 0.45438897609710693, 11),
 ((35, 8, 'sigmoid'), 0.4545985758304596, 14),
 ((70, 7, 'sigmoid'), 0.45485538244247437, 11),
 ((35, 4, 'softmax'), 0.45490142703056335, 24),
 ((70, 5, 'sigmoid'), 0.45498475432395935, 13),
 ((140, 3, 'sigmoid'), 0.4551302194595337, 25),
 ((140, 4, 'sigmoid'), 0.45517075061798096, 12),
 ((35, 7, 'sigmoid'), 0.4554622769355774, 16),
 ((35, 4, 'sigmoid'), 0.4555053114891052, 17),
 ((280, 8, 'softsign'), 0.45580723881721497, 8),
 ((280, 3, 'softmax'), 0.4561631381511688, 28),
 ((70, 8, 'softsign'), 0.4561786651611328, 8),
 ((280, 6, 'softsign'), 0.456244468688

In [19]:
test_model(140, 5, 'softsign', 1)

Epoch 1/100
406/406 [==============================] - 3s 7ms/step - loss: 0.4836 - val_loss: 0.4711
Epoch 2/100
406/406 [==============================] - 2s 6ms/step - loss: 0.4613 - val_loss: 0.4645
Epoch 3/100
406/406 [==============================] - 2s 6ms/step - loss: 0.4491 - val_loss: 0.4651
Epoch 4/100
406/406 [==============================] - 2s 6ms/step - loss: 0.4497 - val_loss: 0.4587
Epoch 5/100
406/406 [==============================] - 2s 6ms/step - loss: 0.4392 - val_loss: 0.4642
Epoch 6/100
406/406 [==============================] - 3s 6ms/step - loss: 0.4377 - val_loss: 0.4616
Epoch 7/100
406/406 [==============================] - 3s 6ms/step - loss: 0.4385 - val_loss: 0.4583
Epoch 8/100
406/406 [==============================] - 3s 6ms/step - loss: 0.4326 - val_loss: 0.4644
Epoch 9/100
406/406 [==============================] - 2s 6ms/step - loss: 0.4292 - val_loss: 0.4647
Epoch 10/100
406/406 [==============================] - 2s 6ms/step - loss: 0.4326 - val_lo

(0.45827582478523254,
 12,
 <keras.engine.sequential.Sequential at 0x257a61a94c8>)